In [1]:
# Nescessary imports

%matplotlib inline

# supress warnings
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # supress tensorflow warnings


# import libraries
import sys
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split

# set random seed
random.seed(42)

%load_ext autoreload

In [2]:
%autoreload 2
from data_wraper import Dataset

In [3]:
# mount google drive if using google colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if 'google.colab' in sys.modules:
    root_dir = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'Colab Notebooks')
else:
    root_dir = os.getcwd()

if 'google.colab' in sys.modules:
    data_dir = os.path.join(root_dir, 'data')
else:
    data_dir = os.path.join(root_dir, 'data')

In [5]:
paths = {
    'FOVfiting2' : os.path.join(data_dir, 'mapsCeline_FOVfitting_2_pol-60.mat'),
    '3quadrants2' : os.path.join(data_dir, 'mapsCeline_3quadrants_2_pol-60.mat'),
    'FOVfiting3' : os.path.join(data_dir, 'mapsCeline_FOVfitting_3_pol-60.mat'),
    '3quadrants3' : os.path.join(data_dir, 'mapsCeline_3quadrants_3_pol-60.mat'),
    'FOVfiting4' : os.path.join(data_dir, 'mapsCeline_FOVfitting_4_pol-60.mat'),
    '3quadrants4' : os.path.join(data_dir, 'mapsCeline_3quadrants_4_pol-60.mat'),
    'FOVfiting5' : os.path.join(data_dir, 'mapsCeline_FOVfitting_5_pol-60.mat'),
    '3quadrants5' : os.path.join(data_dir, 'mapsCeline_3quadrants_5_pol-60.mat'),
    'FOVfiting6' : os.path.join(data_dir, 'mapsCeline_FOVfitting_6_pol-60.mat'),
    '3quadrants6' : os.path.join(data_dir, 'mapsCeline_3quadrants_6_pol-60.mat'),   
    'FOVfiting9' : os.path.join(data_dir, 'mapsCeline_FOVfitting_9_pol-60.mat'),
    '3quadrants9' : os.path.join(data_dir, 'mapsCeline_3quadrants_9_pol-60.mat'),
    'FOVfiting9a' : os.path.join(data_dir, 'mapsCeline_FOVfitting_9a_polNP.mat'),
    '3quadrants9a' : os.path.join(data_dir, 'mapsCeline_3quadrants_9a_polNP.mat')
}

In [6]:
#calibration_maps = [paths['FOVfiting2'], paths['FOVfiting3'], paths['FOVfiting4'], paths['FOVfiting5'], paths['FOVfiting6'], paths['FOVfiting9'], paths['3quadrants2'], paths['3quadrants3']]
#calibration_maps = [map for map in paths.values()]
calibration_maps = [paths['FOVfiting2']]
dataset = Dataset(calibration_maps)

Added data from /content/drive/MyDrive/Colab Notebooks/data/mapsCeline_FOVfitting_2_pol-60.mat
New lenght: 297


In [7]:
dataset.preprocess_data()
dataset.crop_data(128)

In [8]:
X = np.stack((dataset.map1_list, dataset.map2_list, dataset.map3_list), axis=-1)
y = dataset.combined_list

In [9]:
print(np.min(X), np.max(X))

0.0 0.9999999997128103


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)

def ssim_mse_loss(y_true, y_pred):
    # Compute SSIM loss
    ssim_loss = 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)
    
    # Compute MSE loss
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    
    # Combine SSIM and MSE losses
    combined_loss = ssim_loss + mse_loss
    
    return combined_loss

In [12]:
%autoreload 2
from UNet import UNet
from Experiment import Experiment
from utils import plot_map, plot_radial_profile

In [13]:
# Perform a grid search on experiments

model = 'UNet'

learning_rates = [0.01, 0.001, 0.0001]
optimisers = ['Adam', 'Adadelta']
losses = ['MSE', 'SSIM', 'SSIM + MSE'] 
sizes = [128]
filters = [[32,64,128], [64,128,256], [128,256,512]]
dropouts = [0.1, 0.2, 0.5]
batch_sizes = [16]
epochs = [1]

if 'google.colab' in sys.modules:
    csv_path = os.path.join(root_dir, 'results', model + '.csv')
else:
    csv_path = os.path.join(root_dir, 'results', model + '.csv')

i = 0
experiments = []
for loss in losses:
    if loss == 'MSE':
        loss = tf.keras.losses.MeanSquaredError()
    elif loss == 'SSIM':
        loss = ssim_loss
    elif loss == 'SSIM + MSE':
        loss = ssim_mse_loss
    for size in sizes:
        input_size = (size, size, 3)
        output_size = (size, size, 1)
        for filter in filters:
          for dropout in dropouts:
            for batch_size in batch_sizes:
              for epoch in epochs:
                for learning_rate in learning_rates:
                  for optimiser_name in optimisers:
                    optimiser = None
                    if optimiser_name == 'Adam':
                      optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate)
                    elif optimiser_name == 'Adadelta':
                      optimiser = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
                    model = "UNet"
                    name = f"Grid Search {i}"

                    experiment = Experiment(model, name, optimiser, learning_rate, loss, input_size, output_size, filter, dropout, epoch, batch_size, csv_path)
                    experiments.append(experiment)
                    i += 1

In [14]:
import threading
import concurrent.futures

mutex = threading.Lock()

def run_experiment(experiment):
    experiment.make(X_train, y_train, X_test, y_test)
    mutex.acquire()
    try:
        print(f"Experiment {experiment.name} finished")
        experiment.save_experiment_csv()
    finally:
        mutex.release()

# Define the maximum number of concurrent threads
max_concurrent_threads = 10

# Create a thread pool executor with the maximum concurrent threads
with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_threads) as executor:
    # Submit the experiments to the executor
    futures = [executor.submit(run_experiment, experiment) for experiment in experiments]

    # Wait for all experiments to complete
    concurrent.futures.wait(futures)

In [15]:
df = pd.read_csv(csv_path)
#df.head(len(df))
df.describe()

FileNotFoundError: ignored